<a href="https://colab.research.google.com/github/pinilDissanayaka/American-History-RAG-Application/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq
from google.colab import userdata

In [33]:
os.environ['PINECORN_API_KEY']=userdata.get('PINECORN_API_KEY')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
pinecone=Pinecone(api_key=os.environ['PINECORN_API_KEY'])

In [26]:
INDEX_NAME="american-history"
DIMENSIONS=384

In [27]:
if not INDEX_NAME in pinecone.list_indexes().names():
  pinecone.create_index(
    name=INDEX_NAME,
    dimension=DIMENSIONS,
    metric="euclidean",
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)
else:
  print("Index already exists")

Index already exists


In [36]:
llm = ChatGroq(
    temperature=0.8,
    model="llama3-70b-8192"
)

In [37]:
llm.invoke('hi')

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.084435757, 'prompt_time': 0.005351095, 'queue_time': None, 'total_time': 0.089786852}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-4ed9589b-40f6-4f76-9a8c-e97851ebd316-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})